## Getting the Pages that List Speakers

After some rooting around on the TED website, examining the Speakers home page, it looks like there are 86 pages that list TED speakers, with URLS like this: `https://www.ted.com/speakers?page=1`. My plan is to:

1. create a list of pages in a text file, 
2. download the 86 pages, 
3. parse the speakers out,
4. create a second list of pages with the `ted.com/speakers/speaker_name" format`
5. download all those pages
6. parse them into a CSV for KK.

Oh, I hope this works.

There's going to be a lot of files created in this process, so it's important to remember where I am:

In [1]:
% pwd

'/Users/john/Code/tedtalks/data'

In [2]:
# As always, the collected imports for this notebook are at the top:

import re, csv, os
from bs4 import BeautifulSoup

### Step 1: Create a list of pages

In [3]:
# This is just proof of concept. I actually used range(1,87) to get
# the list I needed and pasted it into a text document.
for i in range(1,5):
    print("https://www.ted.com/speakers?page=" + str(i))

https://www.ted.com/speakers?page=1
https://www.ted.com/speakers?page=2
https://www.ted.com/speakers?page=3
https://www.ted.com/speakers?page=4


### Step 2: Download the pages

Okay, the text file is `speaker_index_pages.txt` in the data/speakers/directory. I used the following to download all 86 speaker pages to the indices directory:

    wget -w 2 -i ../speaker_index_pages.txt

Then, being lazy and not wanting to figure out how to parse all the files into one list (see below), I simply concatenated all the files into one, with the plan to use `BeautifulSoup` to run through it.

    cat indices/* > speakers_all_pages.txt

### Step 3: Parse the Speaker URLs Out of the Pages

Inside the HTML, each speaker's profile can be found in the following line:

    <a class="results__result media media--sm-v m4" href="/speakers/ellen_t_hoen">

So we need the `href` attribute for the `results__result` class.

In [4]:
with open('./speakers/speakers_all_pages.txt', 'r') as myfile:
    data = myfile.read()

In [5]:
the_soup = BeautifulSoup(data, "lxml")

In [8]:
type(the_soup)

bs4.BeautifulSoup

In [9]:
speaker_suffix = the_soup.find('a', {'class':'results__result media media--sm-v m4'})
print(speaker_suffix)

<a class="results__result media media--sm-v m4" href="/speakers/ellen_t_hoen">
<div class="media__image media__image--thumb">
<span class="thumb thumb--square"><span class="thumb__sizer"><span class="thumb__tugger"><img alt="" class=" thumb__image" play="false" src="https://pi.tedcdn.com/r/pe.tedcdn.com/images/ted/bcd3208945bf8b311418b64f917b1f38e84a24e8_800x600.jpg?h=191&amp;w=254"/><span class="thumb__aligner"></span></span></span></span>
</div>
<div class="media__message">
<h4 class="h7 m5">
Ellen<br/>'t Hoen</h4>
<p class="p4">
<strong>Medicine law expert</strong>
</p>
</div>
</a>


In [11]:
speaker_suffix = the_soup.find('a', {'class':'results__result'})['href']
print(type(speaker_suffix), speaker_suffix)

<class 'str'> /speakers/ellen_t_hoen


In [13]:
speaker_suffixes = the_soup.findAll('a', {'class':'results__result media media--sm-v m4'})
len(speaker_suffixes)

30

Why is this only returning 30 items? Using search in a plain text editor (Atom) on the same files above, there are 2572 occurrences of **`results__result`**.

```python
# Archived code that was meant to generate URLs from the soup above
suffixes = [i.attrs["href"] for i in speaker_suffixes]
urls = [str("https://www.ted.com"+suffix) for suffix in suffixes]
```

### Using regex to get the speaker suffix

Okay, I'm still not sure what's going on with the truncated input above, but what I am going to try to do below is read the file in as a string and then use regex to grab the speaker suffixes.

In [15]:
suffixes = re.findall(r'/speakers/(.*?)\'>', data)
len(suffixes)

2572

In [16]:
urls = [str("https://www.ted.com/speakers/"+suffix) for suffix in suffixes]

In [17]:
urls[0:5]

['https://www.ted.com/speakers/ellen_t_hoen',
 'https://www.ted.com/speakers/sandra_aamodt',
 'https://www.ted.com/speakers/trevor_aaronson',
 'https://www.ted.com/speakers/chris_abani',
 'https://www.ted.com/speakers/yassmin_abdel_magied']

### Create a second list with the speaker suffixes

In [18]:
with open('./speakers/speaker_urls.txt', 'w') as f:
    for item in urls:
        f.write("%s\n" % item)

**Step 5** was done using **`wget`**: 

```bash
wget -w 2 -i ../speaker_urls.txt
```

And here's the eventual report:

```
FINISHED --2018-11-15 22:05:00--
Total wall clock time: 2h 0m 43s
Downloaded: 2569 files, 91M in 7.0s (13.1 MB/s)
```

### Step 6: Parse the speaker profiles

In [19]:
% cd speakers

/Users/john/Code/tedtalks/data/speakers


In [26]:
% ls

indices/                 speaker_index_pages.txt  speakers.csv
profiles/                speaker_urls.txt         speakers_all_pages.txt


Here's the HTML where things are found:

* **Name**: `<meta name="author" content="Aala El-Khani" />` or `<h1 class="h2 profile-header__name">`
* **Occupation**: `<div class="p2 profile-header__summary">`
* **Intro**: `<div class="profile-intro">`
* **Profile**: `<div class="section section--minor">`

These are all converted into **`BS4`** searches below. (Fingers crossed that these work.)

In [22]:
def parse(soup):
    name = soup.find('h1', {'class' : 'h2 profile-header__name'}).text.strip('\n')
    occupation = soup.find('div', {'class' : 'p2 profile-header__summary'}).text.strip('\n')
    intro = soup.find('div', {'class' : 'profile-intro'}).text.strip('\n')
    profile = soup.find('div', {'class' : 'section section--minor'}).text.strip('\n')
    return name, occupation, intro, profile

In [23]:
def to_csv(pth, out):
    # open file to write to.
    with open(out, "w") as out:
        # create csv.writer
        wr = csv.writer(out)
        # write headers
        wr.writerow(["name", "occupation", "introduction", "profile"])
        # get all our html files
        for html in os.listdir(pth):
            with open(os.path.join(pth, html)) as f:
                # print(html) # prints off name as it goes?
                # parse the file and write the data to a row.
                wr.writerow(parse(BeautifulSoup(f, "lxml")))

In [24]:
# This is the ACTION:
to_csv("./profiles/","speakers.csv")

gary_haugen
rabbi_lord_jonathan_sacks
joe_kowan
allan_adams
peter_tyack
heather_brooke
reed_hastings
oscar_schwartz
marwa_al_sabouni
steven_addis
nizar_ibrahim
ian_bremmer
lisa_harouni
shashi_tharoor
sergey_brin
candy_chang
susan_solomon
jk_rowling
diane_kelly
thelma_golden
john_kasaona
lee_smolin
elon_musk
yossi_vardi
david_logan
christiane_amanpour
ndidi_nwuneli
jonas_gahr_store
larry_brilliant
zaria_forman
marc_abrahams
adam_driver
halla_tomasdottir
sarah_donnelly
courtney_e_martin
joshua_klein
john_gable
john_la_grou
christine_sun_kim
deborah_gordon
bart_knols
einstein_the_parrot
mike_degruy
peter_gabriel
jill_heinerth
haas_hahn
benjamin_wallace
ethan_zuckerman
marco_tempest
pam_warhurst
neri_oxman
cynthia_kenyon
chimamanda_ngozi_adichie
jonathan_marks
hannah_brencher
birke_baehr
abraham_verghese
katie_hinde
sheila_patek
nirmalya_kumar
raj_panjabi
kate_stafford
lauren_sallan
sian_leah_beilock
jimmy_lin
elizabeth_lindsey
manwar_ali
peter_fankhauser
sue_desmond_hellman
anindya_kundu


AttributeError: 'NoneType' object has no attribute 'text'

In [25]:
import pandas as pd

In [31]:
df = pd.read_csv('speakers.csv')

In [32]:
df.head()

,name,occupation,introduction,profile
0,Gary Haugen,Human rights attorney,"As founder of International Justice Mission, G...",While a member of the 1994 United Nations te...
1,Rabbi Lord Jonathan Sacks,Religious leader,"In a world violently polarized by extremists, ...",Rabbi Lord Sacks is one of Judaism's spiritua...
2,Joe Kowan,Musician and graphic designer,"By day he's a graphic designer, and by night J...",Joe Kowan is a Boston-based musician and grap...
3,Allan Adams,Theoretical physicist,Allan Adams is a theoretical physicist working...,Allan Adams is a theoretical physicist workin...
4,Peter Tyack,Behavioral ecologist,Peter Tyack studies the social behavior and ac...,"Peter Tyack, a senior scientist in biology at..."
